**Import Libraries**

In [1]:
# import Libraries
import os
import pandas as pd
import numpy as np
import requests
from PIL import Image
from io import BytesIO
import cv2

**Download Images**

In [2]:
# Directory to save images
IMAGE_DIR = 'img/train'

# Function to download an image
def download_image(url, image_path):
    try:
        response = requests.get(url, timeout=10)
        img = Image.open(BytesIO(response.content))
        img.save(image_path)
    except Exception as e:
        print(f"Error downloading image from {url}: {e}")

# Read the dataset
df = pd.read_csv('dataset/sample_train.csv')

# Download the images
def download_images(df, IMG_DIR):
    # Ensure the image directory exists
    if not os.path.exists(IMAGE_DIR):
        os.makedirs(IMAGE_DIR)
        
    for i, row in df.iterrows():
        image_url = row['image_link']
        image_path = os.path.join(IMG_DIR, f'image_{i}.jpg')
        download_image(image_url, image_path)
    
    print("Images downloaded!")

download_images(df, IMAGE_DIR)

Images downloaded!


**Data Extraction**

In [3]:
import cv2
import numpy as np

def preprocess_image(image_path, img_size=(224, 224)):
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Failed to load image: {image_path}")
        image = cv2.resize(image, img_size)
        image = image / 255.0  # Normalize to [0,1]
        return image
    except Exception as e:
        print(f"Error preprocessing image {image_path}: {e}")
        return np.zeros((img_size[0], img_size[1], 3))  # Return black image as fallback

# Preprocess all images
def getData(df, IMG_DIR):    
    image_data = []
    for i in range(len(df)):
        image_path = os.path.join(IMG_DIR, f'image_{i}.jpg')
        image = preprocess_image(image_path)
        image_data.append(image)

    # Convert to numpy array
    return np.array(image_data)

image_data = getData(df, IMAGE_DIR)
print(f"Image data shape: {image_data.shape}")

Image data shape: (150, 224, 224, 3)


In [4]:
def extract_value_and_unit(entity_value):
    if isinstance(entity_value, str):
        parts = entity_value.split()
        if len(parts) == 2 and parts[0].replace('.', '', 1).isdigit():
            return float(parts[0]), parts[1]
    return None, None

df['value'], df['unit'] = zip(*df['entity_value'].apply(extract_value_and_unit))

# Drop rows with missing or invalid values
df = df.dropna(subset=['value'])

print(df[['entity_value', 'value', 'unit']].head())

     entity_value     value       unit
0      500.0 gram   500.000       gram
1         1.0 cup     1.000        cup
2      0.709 gram     0.709       gram
3      0.709 gram     0.709       gram
4  1400 milligram  1400.000  milligram


**Improved CNN model**

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Define an improved CNN model
def build_improved_model(input_shape):
    model = tf.keras.Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))  # Dropout for regularization
    model.add(Dense(64, activation='relu'))
    
    model.add(Dense(1, activation='linear'))  # Regression output
    
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

# Build and summarize the improved model
input_shape = (224, 224, 3)
improved_model = build_improved_model(input_shape)
improved_model.summary()

C:\Users\91798\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 222, 222, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 109, 109, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 52, 52, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,178,177 (42.64 MB)

 Trainable params: 11,177,729 (42.64 MB)

 Non-trainable params: 448 (1.75 KB)

**Filter Invalid Rows**

In [6]:
# Initialize a new list for images and a corresponding list for valid values
valid_image_data = []
valid_values = []

# Preprocess all images and keep track of valid samples
for i in range(len(df)):
    image_path = os.path.join(IMAGE_DIR, f'image_{i}.jpg')
    image = preprocess_image(image_path)
    
    # Only keep images that are valid (non-zero arrays indicate valid images)
    if np.any(image):
        valid_image_data.append(image)
        valid_values.append(df['value'].iloc[i])

# Convert to numpy arrays
image_data = np.array(valid_image_data)
values = np.array(valid_values)

In [7]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(image_data, df['value'], test_size=0.2, random_state=42)

# Train the improved model
history = improved_model.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val), batch_size=32)

# Evaluate the model on validation data
val_loss, val_mae = improved_model.evaluate(X_val, y_val)
print(f"Validation MAE: {val_mae}")


Epoch 1/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - loss: 273807.0938 - mae: 242.9842 - val_loss: 100366.2031 - val_mae: 162.3305
Epoch 2/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - loss: 169651.4531 - mae: 231.8053 - val_loss: 99707.8984 - val_mae: 160.6980
Epoch 3/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - loss: 248990.6875 - mae: 237.5075 - val_loss: 99956.7031 - val_mae: 161.3149
Epoch 4/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - loss: 125841.4688 - mae: 200.0061 - val_loss: 95726.5000 - val_mae: 152.8080
Epoch 5/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - loss: 122333.8672 - mae: 175.6083 - val_loss: 91282.2891 - val_mae: 148.1165
Epoch 6/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - loss: 80639.7812 - mae: 149.0043 - val_loss: 89384.3047 - val_mae: 147.7918
Epoch 7/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - loss: 51188.4258 - mae: 134.7354 - val_loss: 86428.6719 - val_mae: 147.6526
Epoch 8/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - loss: 73295.1406 - mae: 138.0921 - val_loss: 84453.2500 - val_mae: 14

In [8]:
# testing
predictions = improved_model.predict(X_val)

# Display some results
print(predictions[:5])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 387ms/step
[[110.260155]
 [242.92664 ]
 [ 90.34032 ]
 [ 93.6224  ]
 [405.28537 ]]


**Formating predictions**

In [9]:
# Formating predictions
def format_prediction(value, entity_name):
    # Placeholder logic for units based on entity_name
    if entity_name in ['width', 'height', 'depth']:
        unit = 'inch'
    elif entity_name == 'item_weight':
        unit = 'gram'
    else:
        unit = ''
    return f"{value:.2f} {unit}"

df['prediction'] = df.apply(lambda row: format_prediction(row['value'], row['entity_name']), axis=1)
df.head()

,image_link,group_id,entity_name,entity_value,value,unit,prediction
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram,500.000,gram,500.00 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup,1.000,cup,1.00
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram,0.709,gram,0.71 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram,0.709,gram,0.71 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram,1400.000,milligram,1400.00 gram


In [10]:
df.rename( columns={'Unnamed: 0':'index'}, inplace=True )
df.head()

,image_link,group_id,entity_name,entity_value,value,unit,prediction
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram,500.000,gram,500.00 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup,1.000,cup,1.00
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram,0.709,gram,0.71 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram,0.709,gram,0.71 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram,1400.000,milligram,1400.00 gram


**Make prediction from test dataset**

In [ ]:
IMAGE_DIR = "img/test"

test = pd.read_csv("dataset/test.csv")
test = test.head(30)

def download_and_process(df, IMG_DIR):
    if not os.path.exists(IMG_DIR):
        os.makedirs(IMG_DIR)
    image_data = []
    for i, row in df.iterrows():
        image_url = row['image_link']
        image_path = os.path.join(IMG_DIR, f'image_{i}.jpg')
        download_image(image_url, image_path)
        image = preprocess_image(image_path)
        image_data.append(image)

    return image_data

data = download_and_process(test, IMAGE_DIR)

# convert to numpy array
data = np.array(data)

prediction = improved_model.predict(data)

test['prediction'] = prediction
test['prediction'] = test.apply(lambda row: format_prediction(row['prediction'], row['entity_name']), axis=1)

**Export test output file**

In [ ]:
output_filename = 'dataset/test2_out.csv'
test[['index', 'prediction']].to_csv(output_filename, index=False)
print(f"Predictions saved to {output_filename}")